<a href="https://colab.research.google.com/github/SarsijNayan/PLAGIARIASM_REMOVER/blob/main/Plagrism_REMOVER_NEW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================
# 📌 NLTK-based Plagiarism Remover – Colab Version
# ============================================

!pip install nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import random

stop_words = stopwords.words("english")

def plagiarism_remover(i):
    word = i
    synonyms = []

    # skip stopwords
    if word.lower() in stop_words:
        return word

    # skip tokens not in WordNet
    if wordnet.synsets(word) == []:
        return word

    # get synonyms
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.append(lemma.name())

    # POS tag original word
    pos_tag_word = nltk.pos_tag([word])

    # POS tag synonyms
    pos = []
    for s in synonyms:
        pos.append(nltk.pos_tag([s]))

    final_synonyms = [p[0][0] for p in pos if p[0][1] == pos_tag_word[0][1]]
    final_synonyms = list(set(final_synonyms))

    # no synonym with same POS tag
    if final_synonyms == []:
        return word

    # keep capitalization
    if word.istitle():
        return random.choice(final_synonyms).title()
    else:
        return random.choice(final_synonyms)

def plagiarism_removal(para):
    para_split = word_tokenize(para)
    final_text = [plagiarism_remover(i) for i in para_split]
    final_text = " ".join(final_text)
    return final_text


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
!pip install nltk
import nltk

# Download everything required
nltk.download('punkt')
nltk.download('punkt_tab')    # ← This fixes your error
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [ ]:
# Complete Colab cell — fixed (works in Google Colab)
!pip install -q nltk ipywidgets

import nltk
nltk_data_to_download = [
    'punkt',
    'punkt_tab',
    'averaged_perceptron_tagger',
    'averaged_perceptron_tagger_eng',
    'wordnet',
    'stopwords'
]
for res in nltk_data_to_download:
    try:
        nltk.download(res, quiet=True)
    except Exception as e:
        print(f"Warning downloading {res}: {e}")

import random, re
from nltk.corpus import wordnet, stopwords
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.tokenize.treebank import TreebankWordDetokenizer
import ipywidgets as widgets
from IPython.display import display

_detok = TreebankWordDetokenizer()
_stop_words = set(stopwords.words("english"))

def _is_simple_word(token):
    return bool(re.fullmatch(r"[A-Za-z]+", token))

def _get_synonyms_same_pos(word):
    syns = []
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            name = lemma.name().replace('_', ' ')
            if re.fullmatch(r"[A-Za-z]+", name):
                syns.append(name)
    if not syns:
        return []
    orig_pos = pos_tag([word])[0][1]
    final = []
    for s in set(syns):
        tagged = pos_tag([s])[0][1]
        if tagged == orig_pos:
            final.append(s)
    return final

def plagiarism_remover_token(token):
    if token.lower() in _stop_words:
        return token
    if not _is_simple_word(token):
        return token
    candidates = _get_synonyms_same_pos(token.lower())
    if not candidates:
        return token
    chosen = random.choice(candidates)
    if token.istitle():
        return chosen.title()
    elif token.isupper():
        return chosen.upper()
    else:
        return chosen

def plagiarism_removal(text):
    if not text or not text.strip():
        return text
    tokens = word_tokenize(text)
    replaced = [plagiarism_remover_token(t) for t in tokens]
    return _detok.detokenize(replaced)

# UI widgets
text_input = widgets.Textarea(
    value="",
    placeholder="Type or paste text here...",
    description="Input:",
    layout=widgets.Layout(width='880px', height='200px')
)

output = widgets.Textarea(
    value="",
    placeholder="Plagiarism-removed text will appear here...",
    description="Output:",
    layout=widgets.Layout(width='880px', height='200px')
)

button = widgets.Button(
    description="Remove Plagiarism",
    button_style='primary'
)

seed_box = widgets.Text(
    value="",
    placeholder="(optional integer)",
    description='Random seed:',
    layout=widgets.Layout(width='260px')
)

status = widgets.HTML(value="")

help_html_widget = widgets.HTML(
    "<small>Notes: (1) Press the button multiple times to get different synonym choices. "
    "(2) Leave Random seed blank for true randomness or enter an integer to reproduce an output. "
    "(3) This replaces single-word synonyms only; punctuation, numbers and multi-word synonyms are kept.</small>"
)

def on_click(b):
    # handle seed (allow blank)
    seed_val = seed_box.value.strip()
    if seed_val != "":
        try:
            random.seed(int(seed_val))
            status.value = "<span style='color:green'>Using seed: {}</span>".format(seed_val)
        except Exception:
            random.seed(None)
            status.value = "<span style='color:orange'>Invalid seed — using random.</span>"
    else:
        random.seed(None)
        status.value = "<span style='color:green'>Processing (random seed)…</span>"
    try:
        inp = text_input.value
        out = plagiarism_removal(inp)
        output.value = out
        status.value = "<span style='color:green'>Done — press again for another variation.</span>"
    except Exception as e:
        status.value = f"<span style='color:red'>Error: {e}</span>"

button.on_click(on_click)

layout = widgets.VBox([
    text_input,
    widgets.HBox([button, seed_box, status]),
    output,
    help_html_widget
])

display(layout)
